# Radiomics Feature Extraction

Extracting features from PET and CT images.

In [1]:
import nrrd
import utils 

import numpy as np
import pandas as pd

from feature_extraction import feature_extractor

In [2]:
# Path to images.
path_ct_imagedir = './../../../data_source/images/ct_nrrd/'
path_pet_imagedir = './../../../data_source/images/pet_nrrd/'
path_masksdir = './../../../data_source/images/masks_nrrd/'

In [3]:
paths_ct_images = utils.sample_paths(
    path_ct_imagedir, path_masksdir, target_format='nrrd'
)
paths_pet_images = utils.sample_paths(
    path_pet_imagedir, path_masksdir, target_format='nrrd'
)

## Setup

In [4]:
def bin_widths(path_images, nbins):
    """Average min and max to compare histogram across stacks.
    Fixed bin iwdth (and not fixed bin size) to compare texture
    features across stacks.
    
    """
    
    img_max, img_min = [], []
    for image_path in path_images:
        image, _ = nrrd.read(image_path['Image'])
        img_max.append(np.ceil(np.max(image)))
        img_min.append(np.floor(np.min(image)))
        
    #print(np.mean(img_max))
        
    return (np.mean(img_max) - np.mean(img_min)) / nbins

In [ ]:
bin_widths(paths_ct_images, 32), bin_widths(paths_ct_images, 64), bin_widths(paths_ct_images, 128)

In [ ]:
bin_widths(paths_pet_images, 16)#, bin_widths(paths_pet_images, 64), bin_widths(paths_pet_images, 128)

## Calculate features

In [5]:
ct_param_files = [
    './parameter_files/alise/ct_128_sqroot_config.yaml',
    #'./parameter_files/no_filter/ct_32_no_filter_config.yaml',
    #'./parameter_files/no_filter/ct_64_no_filter_config.yaml',
    #'./parameter_files/no_filter/ct_128_no_filter_config.yaml',
    #'./parameter_files/gauss/ct_32_05gauss_config.yaml',
    #'./parameter_files/gauss/ct_64_05gauss_config.yaml',
    #'./parameter_files/gauss/ct_128_05gauss_config.yaml',
    #'./parameter_files/sqroot/ct_32_sqroot_config.yaml',
    #'./parameter_files/sqroot/ct_64_sqroot_config.yaml',
    #'./parameter_files/sqroot/ct_128_sqroot_config.yaml'
]
ct_feature_files = [
    './../../../data_source/radiomic_features/alise/ct_128_no_filter_config.csv',
    #'./../../../data_source/radiomic_features/no_filter/ct_32_no_filter_config.csv',
    #'./../../../data_source/radiomic_features/no_filter/ct_64_no_filter_config.csv',
    #'./../../../data_source/radiomic_features/no_filter/ct_128_no_filter_config.csv',
    #'./../../../data_source/radiomic_features/gauss/ct_32_05gauss_config.csv',
    #'./../../../data_source/radiomic_features/gauss/ct_64_05gauss_config.csv',
    #'./../../../data_source/radiomic_features/gauss/ct_128_05gauss_config.csv',
    #'./../../../data_source/radiomic_features/sqroot/ct_32_sqroot_config.csv',
    #'./../../../data_source/radiomic_features/sqroot/ct_64_sqroot_config.csv',
    #'./../../../data_source/radiomic_features/sqroot/ct_128_sqroot_config.csv'
]
for ct_param_file, ct_feature_file in zip(ct_param_files, ct_feature_files):
    feature_extractor(
        ct_param_file,
        paths_ct_images, 
        verbose=1, 
        path_to_results=ct_feature_file
    )

Initiated feature extraction.


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   23.3s
[Parallel(n_jobs=3)]: Done 198 out of 198 | elapsed:  1.1min finished


In [6]:
pet_param_files = [
    './parameter_files/alise/pet_16_sqroot_config.yaml',
    #'./parameter_files/no_filter/pet_32_no_filter_config.yaml',
    #'./parameter_files/no_filter/pet_64_no_filter_config.yaml',
    #'./parameter_files/no_filter/pet_128_no_filter_config.yaml'
    #'./parameter_files/gauss/pet_32_05gauss_config.yaml',
    #'./parameter_files/gauss/pet_64_05gauss_config.yaml',
    #'./parameter_files/gauss/pet_128_05gauss_config.yaml',
    #'./parameter_files/sqroot/pet_32_sqroot_config.yaml',
    #'./parameter_files/sqroot/pet_64_sqroot_config.yaml',
    #'./parameter_files/sqroot/pet_128_sqroot_config.yaml'
]
pet_feature_files = [
    './../../../data_source/radiomic_features/alise/pet_16_sqroot_config.csv',
    #'./../../../data_source/radiomic_features/no_filter/pet_32_no_filter_config.csv',
    #'./../../../data_source/radiomic_features/no_filter/pet_64_no_filter_config.csv',
    #'./../../../data_source/radiomic_features/no_filter/pet_128_no_filter_config.csv'
    #'./../../../data_source/radiomic_features/gauss/pet_32_05gauss_config.csv',
    #'./../../../data_source/radiomic_features/gauss/pet_64_05gauss_config.csv',
    #'./../../../data_source/radiomic_features/gauss/pet_128_05gauss_config.csv',
    #'./../../../data_source/radiomic_features/sqroot/pet_32_sqroot_config.csv',
    #'./../../../data_source/radiomic_features/sqroot/pet_64_sqroot_config.csv',
    #'./../../../data_source/radiomic_features/sqroot/pet_128_sqroot_config.csv'
]
for pet_param_file, pet_feature_file in zip(pet_param_files, pet_feature_files):
    feature_extractor(
        pet_param_file,
        paths_pet_images, 
        verbose=1, 
        path_to_results=pet_feature_file
    )

Initiated feature extraction.


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    9.6s
[Parallel(n_jobs=3)]: Done 198 out of 198 | elapsed:   46.6s finished
